# Домашнее задание по теме «Рекомендации на основе содержания»


    1 Использовать dataset MovieLens
    2 Построить рекомендации (регрессия, предсказываем оценку) на фичах:

    TF-IDF на тегах и жанрах
    Средние оценки (+ median, variance, etc.) пользователя и фильма

    3 Оценить RMSE на тестовой выборке



In [168]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [169]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [312]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [171]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [172]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [173]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [174]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [175]:
movies_with_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaN
9739,193585,Flint (2017),Drama,NaN,NaN,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaN


In [176]:
# movies_with_tags.dropna(subset=['userId'])

In [177]:
movies_with_tags.dropna(subset=['tag'], inplace=True)

In [178]:
# movies_with_tags[movies_with_tags['userId'].isna()]

In [179]:
# movies_with_tags.dropna()

TF-IDF на жанрах

In [180]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [181]:
movie_genres = [change_string(g) for g in movies_with_tags.genres.values]

In [182]:
# len(movie_genres)

In [183]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [184]:
# X_train_counts.shape

In [185]:
# X_train_counts.todense()

In [186]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [190]:
pd.DataFrame(X_train_tfidf.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.408815,0.454788,0.000000,0.656878,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.441078,0.0,0.0,0.0
3679,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3680,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3681,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0


In [289]:
# pd.DataFrame(X_train_tfidf.todense()).columns

In [292]:
# функция для переименования столбцов
def ren_col(columns, prefix):
    names = []
    for p in list(columns):
        names.append(prefix+'_{}'.format(p))
#         print(p)
    return names

In [290]:
# поменяем названия столбцов

In [293]:
prefix='gen'
columns= pd.DataFrame(X_train_tfidf.todense()).columns
g_columns = ren_col(pd.DataFrame(X_train_tfidf.todense()).columns, prefix)

In [294]:
tf_g = pd.DataFrame(X_train_tfidf.todense())
tf_g.columns = g_columns
tf_g

,gen_0,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,gen_10,gen_11,gen_12,gen_13,gen_14,gen_15,gen_16,gen_17,gen_18,gen_19
0,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.408815,0.454788,0.000000,0.656878,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.441078,0.0,0.0,0.0
3679,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3680,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3681,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0


TF-IDF на тэгах

In [295]:
# movies_with_tags.tag

In [296]:
movie_tags = [change_string(t) for t in movies_with_tags.tag.values]

In [297]:
# len(movie_tags)

In [298]:
count_vect_t = CountVectorizer()
X_train_counts_t = count_vect_t.fit_transform(movie_tags)

In [299]:
# X_train_counts_t.shape

In [300]:
# X_train_counts_t.todense()

In [301]:
tfidf_transformer_t = TfidfTransformer()
X_train_tfidf_t = tfidf_transformer_t.fit_transform(X_train_counts_t)

In [302]:
pd.DataFrame(X_train_tfidf_t.todense())

,0,1,2,3,4,5,6,7,8,9,...,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [303]:
# переименуем столбцы

In [305]:
prefix='tag'
columns= pd.DataFrame(X_train_tfidf_t.todense()).columns
t_columns = ren_col(pd.DataFrame(X_train_tfidf_t.todense()), prefix)

In [307]:
tf_t = pd.DataFrame(X_train_tfidf_t.todense())
tf_t.columns = t_columns
tf_t

,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,...,tag_1462,tag_1463,tag_1464,tag_1465,tag_1466,tag_1467,tag_1468,tag_1469,tag_1470,tag_1471
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Объединим в одну таблицу

In [313]:
movies_with_tags.join(tf_g.join(tf_t))

,movieId,title,genres,userId,tag,timestamp,gen_0,gen_1,gen_2,gen_3,...,tag_1462,tag_1463,tag_1464,tag_1465,tag_1466,tag_1467,tag_1468,tag_1469,tag_1470,tag_1471
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,0.0,0.374094,0.507479,0.540326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09,0.0,0.374094,0.507479,0.540326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09,0.0,0.374094,0.507479,0.540326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09,0.0,0.374094,0.507479,0.540326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09,0.0,0.374094,0.507479,0.540326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9710,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,1.528935e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,1.537099e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,1.537099e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,1.537099e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
movies_with_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09
...,...,...,...,...,...,...
9710,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,1.528935e+09
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,1.537099e+09
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,1.537099e+09
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,1.537099e+09


In [216]:
# movies_tfidf = movies_with_tags.join(pd.DataFrame(X_train_tfidf_t.todense()))

In [217]:
# movies_tfidf

In [218]:
# movies_tfidf.join(pd.DataFrame(X_train_tfidf.todense()))

In [229]:
pd.DataFrame(X_train_tfidf.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.374094,0.507479,0.540326,0.292786,0.0,0.0,0.0,0.474177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.461618,0.000000,0.666743,0.000000,0.0,0.0,0.0,0.585117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.408815,0.454788,0.000000,0.656878,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.441078,0.0,0.0,0.0
3679,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3680,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0
3681,0.438549,0.000000,0.661817,0.000000,0.381831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473159,0.0,0.0,0.0


In [231]:
pd.DataFrame(X_train_tfidf_t.todense())

,0,1,2,3,4,5,6,7,8,9,...,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Найдем пользователя, который поставил наибольшее количество оценок

In [22]:
ratings.groupby('userId').count().sort_values('rating', ascending=False)

,movieId,rating,timestamp
userId,,,
414,2698,2698,2698
599,2478,2478,2478
474,2108,2108,2108
448,1864,1864,1864
274,1346,1346,1346
...,...,...,...
442,20,20,20
569,20,20,20
320,20,20,20


In [23]:
tags.groupby('userId').count().sort_values('tag', ascending=False).head(1)

,movieId,tag,timestamp
userId,,,
474,1507,1507,1507


In [25]:
movies_with_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaN
9739,193585,Flint (2017),Drama,NaN,NaN,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaN


Возьмем пользователя с id 474

In [26]:
data = movies_with_tags[movies_with_tags['userId'] == 474]
data

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,474.0,game,1.137376e+09
4,5,Father of the Bride Part II (1995),Comedy,474.0,pregnancy,1.137374e+09
4,5,Father of the Bride Part II (1995),Comedy,474.0,remake,1.137374e+09
6,7,Sabrina (1995),Comedy|Romance,474.0,remake,1.137376e+09
...,...,...,...,...,...,...
6063,40819,Walk the Line (2005),Drama|Musical|Romance,474.0,Johnny Cash,1.137201e+09
6075,41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,474.0,C.S. Lewis,1.137182e+09
6087,41997,Munich (2005),Action|Crime|Drama|Thriller,474.0,In Netflix queue,1.137180e+09
6088,42002,"Producers, The (2005)",Comedy|Musical,474.0,In Netflix queue,1.137202e+09


In [27]:
data.describe(include='object')

,title,genres,tag
count,1507,1507,1507
unique,1235,287,595
top,Dark Days (2000),Drama,In Netflix queue
freq,6,266,131


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1507 entries, 0 to 6114
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    1507 non-null   int64  
 1   title      1507 non-null   object 
 2   genres     1507 non-null   object 
 3   userId     1507 non-null   float64
 4   tag        1507 non-null   object 
 5   timestamp  1507 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 82.4+ KB


In [28]:
# data.dropna(inplace=True)

In [29]:
# data.info()

In [47]:
data.groupby('title').count()

,movieId,genres,userId,tag,timestamp
title,,,,,
...And Justice for All (1979),1,1,1,1,1
10 Things I Hate About You (1999),1,1,1,1,1
101 Dalmatians (1996),2,2,2,2,2
101 Dalmatians (One Hundred and One Dalmatians) (1961),1,1,1,1,1
"11'09""01 - September 11 (2002)",1,1,1,1,1
...,...,...,...,...,...
Z (1969),1,1,1,1,1
Zelary (2003),1,1,1,1,1
Zelig (1983),1,1,1,1,1


In [48]:
tag_strings = []
movies = []

for movie, group in tqdm_notebook(data.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies.append(movie)

C:\Users\FED\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [38]:
len(tag_strings)

1235

In [39]:
len(movies)

1235

In [50]:
tag_strings[:5]

['lawyers', 'Shakespearesortof', 'dogs remake', 'Disney', 'terrorism']

In [51]:
movies[:5]

['...And Justice for All (1979)',
 '10 Things I Hate About You (1999)',
 '101 Dalmatians (1996)',
 '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
 '11\'09"01 - September 11 (2002)']

In [54]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [55]:
movie_genres = [change_string(g) for g in data.genres.values]

In [57]:
len(movie_genres)

1507